# Chapter 10: Exporting Structures to File Formats

## Learning Objectives
- Export atomic structures to common file formats
- Understand format specifications and conventions
- Read structures back from files
- Convert between different formats

---

## 10.1 Overview of Common File Formats

| Format | Extension | Periodicity | Common Use |
|--------|-----------|-------------|------------|
| XYZ | .xyz | No | Molecules, visualization |
| PDB | .pdb | No | Biomolecules |
| CIF | .cif | Yes | Crystallography |
| POSCAR | POSCAR | Yes | VASP calculations |
| LAMMPS | .data | Yes | MD simulations |

Each format has specific conventions and strengths.

In [ ]:
import numpy as np
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass
from datetime import datetime
import os

# Structure classes for this chapter
@dataclass
class LatticeParameters:
    a: float
    b: float
    c: float
    alpha: float = 90.0
    beta: float = 90.0
    gamma: float = 90.0

def lattice_vectors_from_parameters(params: LatticeParameters) -> np.ndarray:
    alpha = np.radians(params.alpha)
    beta = np.radians(params.beta)
    gamma = np.radians(params.gamma)
    
    a_vec = np.array([params.a, 0, 0])
    bx = params.b * np.cos(gamma)
    by = params.b * np.sin(gamma)
    b_vec = np.array([bx, by, 0])
    
    cx = params.c * np.cos(beta)
    cy = params.c * (np.cos(alpha) - np.cos(beta)*np.cos(gamma)) / np.sin(gamma)
    cz = np.sqrt(max(0, params.c**2 - cx**2 - cy**2))
    c_vec = np.array([cx, cy, cz])
    
    return np.array([a_vec, b_vec, c_vec])

def parameters_from_lattice_vectors(vectors: np.ndarray) -> LatticeParameters:
    """Extract lattice parameters from vectors."""
    a_vec, b_vec, c_vec = vectors
    a = np.linalg.norm(a_vec)
    b = np.linalg.norm(b_vec)
    c = np.linalg.norm(c_vec)
    
    alpha = np.degrees(np.arccos(np.dot(b_vec, c_vec) / (b * c)))
    beta = np.degrees(np.arccos(np.dot(a_vec, c_vec) / (a * c)))
    gamma = np.degrees(np.arccos(np.dot(a_vec, b_vec) / (a * b)))
    
    return LatticeParameters(a, b, c, alpha, beta, gamma)

class Structure:
    """General structure class for atoms."""
    
    def __init__(self, name: str = "structure", 
                 lattice_vectors: np.ndarray = None,
                 is_periodic: bool = False):
        self.name = name
        self.lattice_vectors = lattice_vectors
        self.is_periodic = is_periodic
        self.atoms: List[Tuple[str, np.ndarray]] = []  # (symbol, position)
        self.use_fractional = is_periodic  # Use fractional for periodic systems
    
    def add_atom(self, symbol: str, position: np.ndarray) -> None:
        self.atoms.append((symbol, np.array(position, dtype=float)))
    
    @property
    def n_atoms(self) -> int:
        return len(self.atoms)
    
    @property
    def symbols(self) -> List[str]:
        return [s for s, _ in self.atoms]
    
    @property
    def positions(self) -> np.ndarray:
        return np.array([p for _, p in self.atoms])
    
    def get_cartesian_positions(self) -> np.ndarray:
        """Get positions in Cartesian coordinates."""
        if self.use_fractional and self.lattice_vectors is not None:
            return self.positions @ self.lattice_vectors
        return self.positions
    
    def get_fractional_positions(self) -> np.ndarray:
        """Get positions in fractional coordinates."""
        if not self.use_fractional and self.lattice_vectors is not None:
            return self.positions @ np.linalg.inv(self.lattice_vectors)
        return self.positions

# Create example structures
def build_example_molecule() -> Structure:
    """Build a water molecule."""
    mol = Structure("water", is_periodic=False)
    mol.add_atom('O', [0, 0, 0])
    mol.add_atom('H', [0.757, 0.587, 0])
    mol.add_atom('H', [-0.757, 0.587, 0])
    return mol

def build_example_crystal() -> Structure:
    """Build FCC copper."""
    params = LatticeParameters(3.615, 3.615, 3.615, 90, 90, 90)
    vectors = lattice_vectors_from_parameters(params)
    crystal = Structure("Cu_FCC", vectors, is_periodic=True)
    
    fcc_positions = [[0, 0, 0], [0.5, 0.5, 0], [0.5, 0, 0.5], [0, 0.5, 0.5]]
    for pos in fcc_positions:
        crystal.add_atom('Cu', pos)
    
    return crystal

water = build_example_molecule()
copper = build_example_crystal()
print(f"Water: {water.n_atoms} atoms")
print(f"Copper: {copper.n_atoms} atoms")

## 10.2 XYZ Format

The **XYZ format** is the simplest:
```
N
comment line
element1 x1 y1 z1
element2 x2 y2 z2
...
```

In [ ]:
def write_xyz(structure: Structure, filename: str, comment: str = None) -> str:
    """Write structure to XYZ format.
    
    Args:
        structure: Structure to write
        filename: Output filename
        comment: Optional comment line
    
    Returns:
        XYZ file content as string
    """
    lines = []
    
    # Number of atoms
    lines.append(str(structure.n_atoms))
    
    # Comment line
    if comment is None:
        comment = f"{structure.name} - Generated {datetime.now().strftime('%Y-%m-%d')}"
    lines.append(comment)
    
    # Atom lines
    positions = structure.get_cartesian_positions()
    for symbol, pos in zip(structure.symbols, positions):
        lines.append(f"{symbol:2s} {pos[0]:15.10f} {pos[1]:15.10f} {pos[2]:15.10f}")
    
    content = '\n'.join(lines) + '\n'
    
    if filename:
        with open(filename, 'w') as f:
            f.write(content)
    
    return content

def read_xyz(filename: str) -> Structure:
    """Read structure from XYZ file."""
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    n_atoms = int(lines[0].strip())
    comment = lines[1].strip()
    
    structure = Structure(comment, is_periodic=False)
    
    for i in range(2, 2 + n_atoms):
        parts = lines[i].split()
        symbol = parts[0]
        position = [float(parts[1]), float(parts[2]), float(parts[3])]
        structure.add_atom(symbol, position)
    
    return structure

# Write water to XYZ
xyz_content = write_xyz(water, '', comment="Water molecule")
print("XYZ format:")
print("═" * 40)
print(xyz_content)

In [ ]:
# Extended XYZ with lattice information
def write_extended_xyz(structure: Structure, filename: str) -> str:
    """Write structure to extended XYZ format with lattice.
    
    Extended XYZ includes lattice information in the comment line.
    """
    lines = []
    lines.append(str(structure.n_atoms))
    
    # Extended XYZ comment with lattice
    if structure.is_periodic and structure.lattice_vectors is not None:
        L = structure.lattice_vectors
        lattice_str = ' '.join([f'{x:.10f}' for row in L for x in row])
        comment = f'Lattice="{lattice_str}" Properties=species:S:1:pos:R:3 pbc="T T T"'
    else:
        comment = f'{structure.name}'
    lines.append(comment)
    
    positions = structure.get_cartesian_positions()
    for symbol, pos in zip(structure.symbols, positions):
        lines.append(f"{symbol:2s} {pos[0]:15.10f} {pos[1]:15.10f} {pos[2]:15.10f}")
    
    content = '\n'.join(lines) + '\n'
    
    if filename:
        with open(filename, 'w') as f:
            f.write(content)
    
    return content

# Write copper to extended XYZ
ext_xyz_content = write_extended_xyz(copper, '')
print("Extended XYZ format:")
print("═" * 60)
print(ext_xyz_content)

## 10.3 PDB Format

The **Protein Data Bank (PDB)** format is standard for biomolecules.

In [ ]:
def write_pdb(structure: Structure, filename: str) -> str:
    """Write structure to PDB format.
    
    Uses simplified ATOM records.
    """
    lines = []
    
    # Header
    lines.append(f"HEADER    {structure.name:40s}{datetime.now().strftime('%d-%b-%y').upper()}")
    lines.append(f"TITLE     {structure.name}")
    
    # Crystal info (CRYST1 record) if periodic
    if structure.is_periodic and structure.lattice_vectors is not None:
        params = parameters_from_lattice_vectors(structure.lattice_vectors)
        lines.append(
            f"CRYST1{params.a:9.3f}{params.b:9.3f}{params.c:9.3f}"
            f"{params.alpha:7.2f}{params.beta:7.2f}{params.gamma:7.2f} P 1           1"
        )
    
    # Atom records
    positions = structure.get_cartesian_positions()
    for i, (symbol, pos) in enumerate(zip(structure.symbols, positions), 1):
        # ATOM record format
        # Columns: 1-6 ATOM, 7-11 serial, 13-16 name, 17 altLoc, 18-20 resName,
        #          22 chainID, 23-26 resSeq, 27 iCode, 31-38 x, 39-46 y, 47-54 z,
        #          55-60 occupancy, 61-66 tempFactor, 77-78 element
        atom_name = f"{symbol:>2s}".ljust(4)
        lines.append(
            f"ATOM  {i:5d} {atom_name} MOL A   1    "
            f"{pos[0]:8.3f}{pos[1]:8.3f}{pos[2]:8.3f}"
            f"  1.00  0.00          {symbol:>2s}"
        )
    
    lines.append("END")
    
    content = '\n'.join(lines) + '\n'
    
    if filename:
        with open(filename, 'w') as f:
            f.write(content)
    
    return content

def read_pdb(filename: str) -> Structure:
    """Read structure from PDB file."""
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    structure = Structure(is_periodic=False)
    lattice = None
    
    for line in lines:
        if line.startswith('TITLE'):
            structure.name = line[10:].strip()
        
        elif line.startswith('CRYST1'):
            a = float(line[6:15])
            b = float(line[15:24])
            c = float(line[24:33])
            alpha = float(line[33:40])
            beta = float(line[40:47])
            gamma = float(line[47:54])
            params = LatticeParameters(a, b, c, alpha, beta, gamma)
            structure.lattice_vectors = lattice_vectors_from_parameters(params)
            structure.is_periodic = True
        
        elif line.startswith('ATOM') or line.startswith('HETATM'):
            symbol = line[76:78].strip()
            if not symbol:
                symbol = line[12:14].strip()
            x = float(line[30:38])
            y = float(line[38:46])
            z = float(line[46:54])
            structure.add_atom(symbol, [x, y, z])
    
    return structure

# Write water to PDB
pdb_content = write_pdb(water, '')
print("PDB format:")
print("═" * 60)
print(pdb_content)

## 10.4 CIF Format

**Crystallographic Information Framework (CIF)** is the standard for crystal structures.

In [ ]:
def write_cif(structure: Structure, filename: str) -> str:
    """Write structure to CIF format."""
    if not structure.is_periodic:
        raise ValueError("CIF format requires periodic structure")
    
    lines = []
    
    # Data block
    safe_name = structure.name.replace(' ', '_')
    lines.append(f"data_{safe_name}")
    lines.append("")
    
    # Cell parameters
    params = parameters_from_lattice_vectors(structure.lattice_vectors)
    lines.append(f"_cell_length_a    {params.a:.6f}")
    lines.append(f"_cell_length_b    {params.b:.6f}")
    lines.append(f"_cell_length_c    {params.c:.6f}")
    lines.append(f"_cell_angle_alpha {params.alpha:.4f}")
    lines.append(f"_cell_angle_beta  {params.beta:.4f}")
    lines.append(f"_cell_angle_gamma {params.gamma:.4f}")
    lines.append("")
    
    # Space group (P1 for general)
    lines.append("_symmetry_space_group_name_H-M 'P 1'")
    lines.append("_symmetry_Int_Tables_number    1")
    lines.append("")
    
    # Atom site loop
    lines.append("loop_")
    lines.append("_atom_site_label")
    lines.append("_atom_site_type_symbol")
    lines.append("_atom_site_fract_x")
    lines.append("_atom_site_fract_y")
    lines.append("_atom_site_fract_z")
    lines.append("_atom_site_occupancy")
    
    # Get fractional positions
    if structure.use_fractional:
        frac_positions = structure.positions
    else:
        frac_positions = structure.get_fractional_positions()
    
    # Count atoms by element for labeling
    element_counts = {}
    for symbol, frac in zip(structure.symbols, frac_positions):
        element_counts[symbol] = element_counts.get(symbol, 0) + 1
        label = f"{symbol}{element_counts[symbol]}"
        lines.append(
            f"{label:6s} {symbol:2s} "
            f"{frac[0]:10.6f} {frac[1]:10.6f} {frac[2]:10.6f} 1.0"
        )
    
    content = '\n'.join(lines) + '\n'
    
    if filename:
        with open(filename, 'w') as f:
            f.write(content)
    
    return content

def read_cif(filename: str) -> Structure:
    """Read structure from CIF file (simplified parser)."""
    with open(filename, 'r') as f:
        content = f.read()
    
    # Extract cell parameters
    import re
    
    def get_value(key):
        match = re.search(rf'{key}\s+([\d.]+)', content)
        return float(match.group(1)) if match else None
    
    a = get_value('_cell_length_a')
    b = get_value('_cell_length_b')
    c = get_value('_cell_length_c')
    alpha = get_value('_cell_angle_alpha') or 90.0
    beta = get_value('_cell_angle_beta') or 90.0
    gamma = get_value('_cell_angle_gamma') or 90.0
    
    params = LatticeParameters(a, b, c, alpha, beta, gamma)
    vectors = lattice_vectors_from_parameters(params)
    
    structure = Structure("from_cif", vectors, is_periodic=True)
    
    # Find atom_site loop and parse
    lines = content.split('\n')
    in_atom_loop = False
    column_order = []
    
    for line in lines:
        line = line.strip()
        
        if line.startswith('loop_'):
            in_atom_loop = False
            column_order = []
        
        if line.startswith('_atom_site_'):
            in_atom_loop = True
            column_order.append(line)
        
        elif in_atom_loop and line and not line.startswith('_'):
            if line.startswith('loop_') or line.startswith('#'):
                in_atom_loop = False
                continue
            
            parts = line.split()
            if len(parts) >= len(column_order):
                # Find indices
                try:
                    sym_idx = column_order.index('_atom_site_type_symbol')
                except ValueError:
                    sym_idx = column_order.index('_atom_site_label')
                x_idx = column_order.index('_atom_site_fract_x')
                y_idx = column_order.index('_atom_site_fract_y')
                z_idx = column_order.index('_atom_site_fract_z')
                
                symbol = ''.join(c for c in parts[sym_idx] if c.isalpha())
                frac = [
                    float(parts[x_idx].split('(')[0]),
                    float(parts[y_idx].split('(')[0]),
                    float(parts[z_idx].split('(')[0])
                ]
                structure.add_atom(symbol, frac)
    
    return structure

# Write copper to CIF
cif_content = write_cif(copper, '')
print("CIF format:")
print("═" * 50)
print(cif_content)

## 10.5 VASP POSCAR Format

**POSCAR** is the structure file format for VASP (Vienna Ab initio Simulation Package).

In [ ]:
def write_poscar(structure: Structure, filename: str, 
                  direct: bool = True, selective_dynamics: bool = False) -> str:
    """Write structure to VASP POSCAR format.
    
    Args:
        structure: Structure to write
        filename: Output filename
        direct: Use Direct (fractional) coordinates if True, Cartesian if False
        selective_dynamics: Include selective dynamics flags
    """
    if not structure.is_periodic:
        raise ValueError("POSCAR requires periodic structure")
    
    lines = []
    
    # Comment line
    lines.append(structure.name)
    
    # Scaling factor
    lines.append("1.0")
    
    # Lattice vectors
    for vec in structure.lattice_vectors:
        lines.append(f"  {vec[0]:20.16f}  {vec[1]:20.16f}  {vec[2]:20.16f}")
    
    # Group atoms by element
    element_order = []
    element_groups = {}
    
    for symbol, pos in structure.atoms:
        if symbol not in element_groups:
            element_order.append(symbol)
            element_groups[symbol] = []
        element_groups[symbol].append(pos)
    
    # Element names line (VASP 5+)
    lines.append('   ' + '   '.join(element_order))
    
    # Element counts
    counts = [str(len(element_groups[el])) for el in element_order]
    lines.append('   ' + '   '.join(counts))
    
    # Selective dynamics (optional)
    if selective_dynamics:
        lines.append("Selective dynamics")
    
    # Coordinate type
    lines.append("Direct" if direct else "Cartesian")
    
    # Positions
    for symbol in element_order:
        for pos in element_groups[symbol]:
            if direct:
                # Use fractional coordinates
                if structure.use_fractional:
                    frac = pos
                else:
                    frac = pos @ np.linalg.inv(structure.lattice_vectors)
                coord_str = f"  {frac[0]:20.16f}  {frac[1]:20.16f}  {frac[2]:20.16f}"
            else:
                # Use Cartesian coordinates
                if structure.use_fractional:
                    cart = pos @ structure.lattice_vectors
                else:
                    cart = pos
                coord_str = f"  {cart[0]:20.16f}  {cart[1]:20.16f}  {cart[2]:20.16f}"
            
            if selective_dynamics:
                coord_str += "   T   T   T"  # All atoms movable
            
            lines.append(coord_str)
    
    content = '\n'.join(lines) + '\n'
    
    if filename:
        with open(filename, 'w') as f:
            f.write(content)
    
    return content

def read_poscar(filename: str) -> Structure:
    """Read structure from VASP POSCAR file."""
    with open(filename, 'r') as f:
        lines = [l.strip() for l in f.readlines()]
    
    name = lines[0]
    scale = float(lines[1])
    
    # Lattice vectors
    vectors = []
    for i in range(2, 5):
        vec = [float(x) for x in lines[i].split()]
        vectors.append(vec)
    vectors = np.array(vectors) * scale
    
    # Element names (VASP 5+) and counts
    line_idx = 5
    first_token = lines[5].split()[0]
    
    if first_token.isalpha():
        elements = lines[5].split()
        counts = [int(x) for x in lines[6].split()]
        line_idx = 7
    else:
        counts = [int(x) for x in lines[5].split()]
        elements = [f"X{i+1}" for i in range(len(counts))]  # Unknown elements
        line_idx = 6
    
    # Check for Selective dynamics
    if lines[line_idx].lower().startswith('s'):
        line_idx += 1
    
    # Coordinate type
    is_direct = lines[line_idx].lower().startswith('d')
    line_idx += 1
    
    # Create structure
    structure = Structure(name, vectors, is_periodic=True)
    structure.use_fractional = is_direct
    
    # Read positions
    for element, count in zip(elements, counts):
        for _ in range(count):
            parts = lines[line_idx].split()
            pos = [float(parts[0]), float(parts[1]), float(parts[2])]
            structure.add_atom(element, pos)
            line_idx += 1
    
    return structure

# Write copper to POSCAR
poscar_content = write_poscar(copper, '')
print("POSCAR format:")
print("═" * 50)
print(poscar_content)

## 10.6 LAMMPS Data Format

**LAMMPS data files** describe systems for molecular dynamics simulations.

In [ ]:
def write_lammps_data(structure: Structure, filename: str,
                       atom_style: str = 'atomic') -> str:
    """Write structure to LAMMPS data format.
    
    Args:
        structure: Structure to write
        filename: Output filename
        atom_style: 'atomic', 'charge', or 'full'
    """
    lines = []
    
    # Header
    lines.append(f"LAMMPS data file - {structure.name}")
    lines.append("")
    
    # Atom counts
    lines.append(f"{structure.n_atoms} atoms")
    
    # Atom types
    unique_elements = list(dict.fromkeys(structure.symbols))  # Preserve order
    type_map = {el: i+1 for i, el in enumerate(unique_elements)}
    lines.append(f"{len(unique_elements)} atom types")
    lines.append("")
    
    # Box bounds
    if structure.is_periodic and structure.lattice_vectors is not None:
        a, b, c = structure.lattice_vectors
        
        # LAMMPS uses xlo, xhi, etc.
        # For non-orthogonal boxes, need xy, xz, yz tilts
        xlo, ylo, zlo = 0, 0, 0
        xhi = a[0]
        xy = b[0]
        yhi = b[1]
        xz = c[0]
        yz = c[1]
        zhi = c[2]
        
        lines.append(f"{xlo:.10f} {xhi:.10f} xlo xhi")
        lines.append(f"{ylo:.10f} {yhi:.10f} ylo yhi")
        lines.append(f"{zlo:.10f} {zhi:.10f} zlo zhi")
        
        if abs(xy) > 1e-10 or abs(xz) > 1e-10 or abs(yz) > 1e-10:
            lines.append(f"{xy:.10f} {xz:.10f} {yz:.10f} xy xz yz")
    else:
        # Non-periodic: use bounding box
        positions = structure.get_cartesian_positions()
        min_pos = np.min(positions, axis=0) - 10
        max_pos = np.max(positions, axis=0) + 10
        lines.append(f"{min_pos[0]:.6f} {max_pos[0]:.6f} xlo xhi")
        lines.append(f"{min_pos[1]:.6f} {max_pos[1]:.6f} ylo yhi")
        lines.append(f"{min_pos[2]:.6f} {max_pos[2]:.6f} zlo zhi")
    
    lines.append("")
    
    # Masses (approximate atomic masses)
    MASSES = {
        'H': 1.008, 'C': 12.011, 'N': 14.007, 'O': 15.999,
        'Cu': 63.546, 'Fe': 55.845, 'Au': 196.967, 'Si': 28.086,
    }
    lines.append("Masses")
    lines.append("")
    for el in unique_elements:
        mass = MASSES.get(el, 1.0)
        lines.append(f"{type_map[el]} {mass:.4f}  # {el}")
    lines.append("")
    
    # Atoms section
    lines.append("Atoms  # atomic")
    lines.append("")
    
    positions = structure.get_cartesian_positions()
    for i, (symbol, pos) in enumerate(zip(structure.symbols, positions), 1):
        atom_type = type_map[symbol]
        if atom_style == 'atomic':
            lines.append(f"{i} {atom_type} {pos[0]:.10f} {pos[1]:.10f} {pos[2]:.10f}")
        elif atom_style == 'charge':
            lines.append(f"{i} {atom_type} 0.0 {pos[0]:.10f} {pos[1]:.10f} {pos[2]:.10f}")
    
    content = '\n'.join(lines) + '\n'
    
    if filename:
        with open(filename, 'w') as f:
            f.write(content)
    
    return content

# Write copper to LAMMPS
lammps_content = write_lammps_data(copper, '')
print("LAMMPS data format:")
print("═" * 50)
print(lammps_content)

## 10.7 Format Converter

In [ ]:
class StructureIO:
    """Unified interface for reading/writing structures."""
    
    WRITERS = {
        'xyz': write_xyz,
        'extxyz': write_extended_xyz,
        'pdb': write_pdb,
        'cif': write_cif,
        'poscar': write_poscar,
        'vasp': write_poscar,
        'lammps': write_lammps_data,
    }
    
    READERS = {
        'xyz': read_xyz,
        'pdb': read_pdb,
        'cif': read_cif,
        'poscar': read_poscar,
        'vasp': read_poscar,
    }
    
    @classmethod
    def write(cls, structure: Structure, filename: str, 
              fmt: str = None, **kwargs) -> str:
        """Write structure to file.
        
        Args:
            structure: Structure to write
            filename: Output filename
            fmt: Format (auto-detected from extension if not provided)
            **kwargs: Format-specific options
        """
        if fmt is None:
            ext = os.path.splitext(filename)[1].lower().strip('.')
            if ext in ['xyz', 'pdb', 'cif']:
                fmt = ext
            elif 'poscar' in filename.lower() or 'contcar' in filename.lower():
                fmt = 'poscar'
            elif ext == 'data':
                fmt = 'lammps'
            else:
                fmt = 'xyz'
        
        writer = cls.WRITERS.get(fmt.lower())
        if writer is None:
            raise ValueError(f"Unknown format: {fmt}")
        
        return writer(structure, filename, **kwargs)
    
    @classmethod
    def read(cls, filename: str, fmt: str = None) -> Structure:
        """Read structure from file."""
        if fmt is None:
            ext = os.path.splitext(filename)[1].lower().strip('.')
            if ext in ['xyz', 'pdb', 'cif']:
                fmt = ext
            elif 'poscar' in filename.lower() or 'contcar' in filename.lower():
                fmt = 'poscar'
            else:
                fmt = 'xyz'
        
        reader = cls.READERS.get(fmt.lower())
        if reader is None:
            raise ValueError(f"Unknown format: {fmt}")
        
        return reader(filename)
    
    @classmethod
    def convert(cls, input_file: str, output_file: str,
                input_fmt: str = None, output_fmt: str = None) -> None:
        """Convert between formats."""
        structure = cls.read(input_file, input_fmt)
        cls.write(structure, output_file, output_fmt)
        print(f"Converted {input_file} → {output_file}")

# Usage examples
print("Available formats:")
print(f"  Write: {list(StructureIO.WRITERS.keys())}")
print(f"  Read:  {list(StructureIO.READERS.keys())}")

## 10.8 Summary: Format Comparison

In [ ]:
# Generate all formats for comparison
print("\n" + "="*70)
print("COPPER (FCC) IN DIFFERENT FORMATS")
print("="*70)

formats = [
    ('XYZ (Extended)', write_extended_xyz),
    ('PDB', write_pdb),
    ('CIF', write_cif),
    ('POSCAR', write_poscar),
    ('LAMMPS', write_lammps_data),
]

for name, writer in formats:
    print(f"\n{'─'*20} {name} {'─'*20}")
    try:
        content = writer(copper, '')
        # Show first 20 lines
        lines = content.split('\n')[:20]
        for line in lines:
            print(line)
        if len(content.split('\n')) > 20:
            print("...")
    except Exception as e:
        print(f"Error: {e}")

---

## Practice Exercises

### Exercise 10.1: Quantum ESPRESSO Input

Write a function to generate a Quantum ESPRESSO input file.

In [ ]:
# YOUR CODE HERE
def write_qe_input(structure: Structure, filename: str,
                    calculation: str = 'scf',
                    ecutwfc: float = 40.0) -> str:
    """Write Quantum ESPRESSO input file.
    
    QE input has namelists (&CONTROL, &SYSTEM, etc.)
    and cards (ATOMIC_SPECIES, ATOMIC_POSITIONS, etc.)
    """
    # TODO: Implement
    pass

### Exercise 10.2: Trajectory Writer

Write a function to save multiple structures as a trajectory file.

In [ ]:
# YOUR CODE HERE
def write_trajectory_xyz(structures: List[Structure], 
                          filename: str) -> None:
    """Write multiple structures as XYZ trajectory.
    
    XYZ trajectory is simply concatenated XYZ files.
    """
    # TODO: Implement
    pass

### Exercise 10.3: Structure Validator

Write a function that validates a structure file for common issues.

In [ ]:
# YOUR CODE HERE
def validate_structure(structure: Structure) -> List[str]:
    """Check structure for common issues.
    
    Returns:
        List of warning messages
    
    Checks:
    - Atoms too close together
    - Atoms outside unit cell (for periodic)
    - Invalid lattice vectors (zero volume, etc.)
    """
    # TODO: Implement
    pass

---

## Key Takeaways

1. **XYZ** is simplest: just coordinates, great for molecules
2. **PDB** includes connectivity, standard for biomolecules
3. **CIF** is comprehensive: symmetry, bibliographic info
4. **POSCAR** is efficient for periodic systems (VASP)
5. **LAMMPS data** includes force field information

## Course Summary

Congratulations! You've completed the course on atomic structure manipulation. You now have the tools to:

1. **Represent atoms** in 3D space with proper coordinates
2. **Apply transformations** using linear algebra (rotation, translation, strain)
3. **Work with symmetry** and point groups
4. **Build crystals** from lattices and bases
5. **Use Miller indices** to describe planes and directions
6. **Create surface slabs** for computational studies
7. **Manipulate molecules** and their conformations
8. **Transform unit cells** and build supercells
9. **Export structures** to any common format

These skills form the foundation for computational chemistry and materials science!